In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
v = pd.read_csv("VuelosAgregadosMensual.csv")

In [3]:
v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321409 entries, 0 to 321408
Data columns (total 41 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   PASSENGERS             321409 non-null  float64
 1   FREIGHT                321409 non-null  float64
 2   MAIL                   321409 non-null  float64
 3   DISTANCE               321409 non-null  float64
 4   UNIQUE_CARRIER         321409 non-null  object 
 5   AIRLINE_ID             321409 non-null  int64  
 6   UNIQUE_CARRIER_NAME    321409 non-null  object 
 7   UNIQUE_CARRIER_ENTITY  321409 non-null  object 
 8   REGION                 321409 non-null  object 
 9   CARRIER                321409 non-null  object 
 10  CARRIER_NAME           321409 non-null  object 
 11  CARRIER_GROUP          321409 non-null  int64  
 12  CARRIER_GROUP_NEW      321409 non-null  int64  
 13  ORIGIN_AIRPORT_ID      321409 non-null  int64  
 14  ORIGIN_AIRPORT_SEQ_ID  321409 non-nu

In [4]:
v = v.rename(columns={"CLASS":"CLASE"})

In [6]:
v.columns

Index(['PASSENGERS', 'FREIGHT', 'MAIL', 'DISTANCE', 'UNIQUE_CARRIER',
       'AIRLINE_ID', 'UNIQUE_CARRIER_NAME', 'UNIQUE_CARRIER_ENTITY', 'REGION',
       'CARRIER', 'CARRIER_NAME', 'CARRIER_GROUP', 'CARRIER_GROUP_NEW',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID',
       'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS',
       'ORIGIN_STATE_NM', 'ORIGIN_COUNTRY', 'ORIGIN_COUNTRY_NAME',
       'ORIGIN_WAC', 'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID',
       'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
       'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_COUNTRY', 'DEST_COUNTRY_NAME',
       'DEST_WAC', 'YEAR', 'QUARTER', 'MONTH', 'DISTANCE_GROUP', 'CLASE',
       'DATA_SOURCE'],
      dtype='object')

In [7]:
v = v.rename(lambda colname: colname.lower(), axis=1)

In [8]:
v.columns

Index(['passengers', 'freight', 'mail', 'distance', 'unique_carrier',
       'airline_id', 'unique_carrier_name', 'unique_carrier_entity', 'region',
       'carrier', 'carrier_name', 'carrier_group', 'carrier_group_new',
       'origin_airport_id', 'origin_airport_seq_id', 'origin_city_market_id',
       'origin', 'origin_city_name', 'origin_state_abr', 'origin_state_fips',
       'origin_state_nm', 'origin_country', 'origin_country_name',
       'origin_wac', 'dest_airport_id', 'dest_airport_seq_id',
       'dest_city_market_id', 'dest', 'dest_city_name', 'dest_state_abr',
       'dest_state_fips', 'dest_state_nm', 'dest_country', 'dest_country_name',
       'dest_wac', 'year', 'quarter', 'month', 'distance_group', 'clase',
       'data_source'],
      dtype='object')

In [19]:
v_f = v.query('clase=="F"')\
.groupby("unique_carrier_name").passengers.sum()\
.nlargest(5)

In [20]:
v_f

unique_carrier_name
Southwest Airlines Co.    162681011.0
Delta Air Lines Inc.      162260114.0
American Airlines Inc.    155782611.0
United Air Lines Inc.     116212143.0
JetBlue Airways            42830602.0
Name: passengers, dtype: float64